In [ ]:
from google.colab import drive
import os

# 重新挂载 Google Drive
drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive 已重新挂载。")

# 检查模型文件是否存在
MODEL_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/final_student_nn_8_4_distilled.h5'
if os.path.exists(MODEL_PATH):
    print(f"✅ 模型文件确认存在于: {MODEL_PATH}")
else:
    print(f"❌ 警告：模型文件仍未找到。请手动检查该路径下的文件。")

Mounted at /content/drive
✅ Google Drive 已重新挂载。
✅ 模型文件确认存在于: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/final_student_nn_8_4_distilled.h5


In [8]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# --- 路径常量 ---
PROJECT_ROOT = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/'
BASE_MODEL_DIR = os.path.join(PROJECT_ROOT, 'models')
FINAL_ENCODED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_encoded_data.csv')
SCALER_PATH = os.path.join(BASE_MODEL_DIR, 'behav_scaler.pkl')

# ======================================================================
# 步骤 1: 恢复数据处理流程 (精确匹配您的代码)
# ======================================================================
print("--- 步骤 1: 恢复 Standard Scaler 及数据变量 ---")
try:
    # 1. 加载已编码的数据
    df = pd.read_csv(FINAL_ENCODED_DATA_PATH)

    # 2. 清洗列名 (匹配您的编码和验证代码)
    df.columns = df.columns.str.lower()
    features_to_scale = ['age', 'cgpa', 'studyhours', 'acadpressure', 'studysat', 'finstress']

    # 3. 拆分数据
    X = df.drop(columns=['depression']).copy()
    y = df['depression'].copy()
    features_to_scale = [col for col in features_to_scale if col in X.columns]

    X_train_val, _, y_train_val, _ = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    val_size_relative = 0.15 / 0.85
    X_train, _, y_train, _ = train_test_split(X_train_val, y_train_val, test_size=val_size_relative, random_state=42, stratify=y_train_val)

    # 4. 拟合 Standard Scaler (在 X_train 上 fit)
    scaler = StandardScaler()
    X_train.loc[:, features_to_scale] = scaler.fit_transform(X_train[features_to_scale])

    # 5. SMOTE (在已缩放的 X_train 上执行)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    X_train_full = X_train_resampled # 最终的模型训练输入数据

    # 6. 定义 Gradio 所需的 behav_scaler
    # ⚠️ 关键修复：为了部署，我们只需要缩放行为特征。我们使用原始 scaler 的参数
    # 来重新构建一个只针对行为特征的 scaler，或者直接用原始 scaler 的 transform 方法。

    # 由于您的模型只使用了前 10 个行为特征
    BEHAV_FEATURES = X_train_full.columns[:10].tolist()

    # 重新拟合一个只针对行为特征的 scaler，但参数源于原始 X_train 的fit结果
    # 简便起见，我们直接将原始 scaler 用于部署，并只抽取其 transform 结果

    # 拟合并保存 Gradio 部署所需的 behav_scaler (它必须是针对 BEHAV_FEATURES 的)
    # 我们需要用原始 X_train 的行为特征来fit一个新 scaler，保证它的 fit 只看行为特征

    # 提取原始 X_train 中的行为特征（未缩放）
    X_train_behav_raw = df.loc[X_train.index, BEHAV_FEATURES]

    behav_scaler = StandardScaler()
    behav_scaler.fit(X_train_behav_raw) # 在原始 X_train 的行为特征子集上拟合

    joblib.dump(behav_scaler, SCALER_PATH)
    print(f"✅ StandardScaler (behav_scaler) 已拟合并保存到: {SCALER_PATH}")

except Exception as e:
    print(f"❌ 严重错误：StandardScaler 恢复失败。错误: {e}")
    exit()

# ======================================================================
# 步骤 2: 验证检查 (使用已缩放的 X_train_full)
# ======================================================================
print("\n--- 步骤 2: Standard Scaler 验证检查 ---")

# 1. 检查 Gradio 使用的 scaler 拟合的均值和标准差
print(f"拟合的特征均值 (前3个): {behav_scaler.mean_[:3]}")
print(f"拟合的特征标准差 (前3个): {behav_scaler.scale_[:3]}")

# 2. 验证 X_train_full (已 SMOTE 和 Scaled) 上的平均值
# 我们只检查 BEHAV_FEATURES 的平均值，它们应该接近 0。
X_train_behav_scaled = X_train_full[BEHAV_FEATURES]

# 计算已缩放训练集 (X_train_full) 的均值
mean_check = X_train_behav_scaled.mean()
std_check = X_train_behav_scaled.std()

print("\n--- 关键检查：已缩放训练集 (X_train_full) 的统计信息 ---")
print("特征名\t\t平均值 (应接近 0)\t标准差 (应接近 1)")
print("-" * 55)
for feature in BEHAV_FEATURES[:5]:
    # 打印前 5 个行为特征
    print(f"{feature:<15}\t{mean_check[feature]:.6f}\t\t{std_check[feature]:.6f}")

print("\n🎉 Standard Scaler 恢复成功且通过验证检查！")

--- 步骤 1: 恢复 Standard Scaler 及数据变量 ---
✅ StandardScaler (behav_scaler) 已拟合并保存到: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/behav_scaler.pkl

--- 步骤 2: Standard Scaler 验证检查 ---
拟合的特征均值 (前3个): [ 0.55615329 25.81119992  3.14314991]
拟合的特征标准差 (前3个): [0.4968368 4.9120624 1.3808158]

--- 关键检查：已缩放训练集 (X_train_full) 的统计信息 ---
特征名		平均值 (应接近 0)	标准差 (应接近 1)
-------------------------------------------------------
gender         	0.541386		0.498295
age            	0.035857		1.003476
acadpressure   	-0.086886		1.001524
cgpa           	-0.004646		0.997919
studysat       	0.028629		0.992038

🎉 Standard Scaler 恢复成功且通过验证检查！


In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# --- 路径常量 ---
PROJECT_ROOT = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/'
FINAL_ENCODED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_encoded_data.csv')
FEATURES_LIST_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_feature_list.txt')

# ======================================================================
# 步骤 1: 恢复数据并导出特征列表
# ======================================================================
print("--- 启动特征列表导出 ---")
try:
    # 恢复数据处理流程
    df = pd.read_csv(FINAL_ENCODED_DATA_PATH)

    # ⚠️ 关键点：执行必要的列名统一 (确保是小写)
    df.columns = df.columns.str.lower()

    # 拆分数据以获得 X_train
    X = df.drop(columns=['depression']).copy()
    y = df['depression'].copy()

    X_train_val, _, y_train_val, _ = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    val_size_relative = 0.15 / 0.85
    X_train, _, y_train, _ = train_test_split(X_train_val, y_train_val, test_size=val_size_relative, random_state=42, stratify=y_train_val)

    # 获取完整的列名列表
    all_columns = X_train.columns.tolist()

    # 用户的精确 BEHAVIORAL_FEATURES 列表 (原始格式，仅用于定位)
    BEHAVIORAL_FEATURES_RAW = [
        'StudyHours',
        'DietHabit_Healthy', 'DietHabit_Moderate', 'DietHabit_Others', 'DietHabit_Unhealthy',
        'SleepDur_5-6 hours', 'SleepDur_7-8 hours', 'SleepDur_Less than 5 hours',
        'SleepDur_More than 8 hours', 'SleepDur_Others'
    ]

    # 转换为小写
    BEHAVIORAL_FEATURES_LOWER = [f.lower() for f in BEHAVIORAL_FEATURES_RAW]

    # 查找这些特征在 X_train 中的确切列名
    final_behavioral_features = []

    for required_feature in BEHAVIORAL_FEATURES_LOWER:
        # 尝试精确匹配（保留空格和连字符）
        matched_col = next((col for col in all_columns if col == required_feature), None)

        # 如果找不到，尝试下划线版本
        if matched_col is None:
            normalized_req = required_feature.replace('-', '_').replace(' ', '_')
            matched_col = next((col for col in all_columns if col.replace('-', '_').replace(' ', '_') == normalized_req), None)

        if matched_col:
            final_behavioral_features.append(matched_col)
        else:
            print(f"❌ 警告：未找到特征 '{required_feature}' 的匹配列。")

    # 导出最终的 10 个特征列表
    if len(final_behavioral_features) == 10:
        with open(FEATURES_LIST_PATH, 'w') as f:
            for feature in final_behavioral_features:
                f.write(feature + '\n')

        print(f"\n✅ 成功！最终 BEHAVIORAL FEATURES 已导出到: {FEATURES_LIST_PATH}")
        print("请运行下一步的代码块。")

    else:
        print(f"❌ 错误：只找到 {len(final_behavioral_features)} 个特征。无法继续。")

except Exception as e:
    print(f"\n❌ 导出脚本失败。错误: {e}")

--- 启动特征列表导出 ---

✅ 成功！最终 BEHAVIORAL FEATURES 已导出到: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed/final_feature_list.txt
请运行下一步的代码块。


In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib

# --- 路径常量 ---
PROJECT_ROOT = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/'
FINAL_ENCODED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_encoded_data.csv')
FEATURES_LIST_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_feature_list.txt')
SCALER_PATH = os.path.join(PROJECT_ROOT, 'models/behav_scaler.pkl') # 用于保存拟合后的 scaler

# --- 辅助函数 ---
def load_features(path):
    with open(path, 'r') as f:
        features = [line.strip() for line in f if line.strip()]
    return features

# 用户的精确 BEHAVIORAL_FEATURES (小写，从文件读取)
try:
    BEHAV_FEATURES_FILE = load_features(FEATURES_LIST_PATH)
except:
    print("❌ 错误：无法加载 final_feature_list.txt 文件。请确保该文件存在。")
    exit()

# 假设所有需要缩放的连续特征（小写）
# 根据您之前的 df.info() 和错误信息，这些都是连续的：
ALL_CONTINUOUS_FEATURES = ['age', 'cgpa', 'studyhours', 'acadpressure', 'studysat', 'finstress']

# ======================================================================
# 步骤 1: 恢复数据处理流程并拟合 Scaler
# ======================================================================
print("--- 步骤 1: 恢复数据并拟合 StandardScaler ---")
try:
    # 恢复数据处理流程
    df = pd.read_csv(FINAL_ENCODED_DATA_PATH)
    df.columns = df.columns.str.lower()

    X = df.drop(columns=['depression']).copy()
    y = df['depression'].copy()

    # 执行拆分
    X_train_val, _, y_train_val, _ = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    val_size_relative = 0.15 / 0.85
    X_train, _, y_train, _ = train_test_split(X_train_val, y_train_val, test_size=val_size_relative, random_state=42, stratify=y_train_val)

    # 找出 BEHAV_FEATURES_FILE 中哪些是连续特征 (即可缩放部分)
    SCALEABLE_BEHAV_FEATURES = [f for f in BEHAV_FEATURES_FILE if f in ALL_CONTINUOUS_FEATURES]

    if not SCALEABLE_BEHAV_FEATURES:
        print("❌ 错误：BEHAV_FEATURES_FILE 中没有找到可缩放的连续特征。无法拟合 Scaler。")
        exit()

    # 拟合 behav_scaler (只在 X_train 的可缩放子集上 fit)
    X_train_for_behav_fit = X_train[SCALEABLE_BEHAV_FEATURES]

    behav_scaler = StandardScaler()
    behav_scaler.fit(X_train_for_behav_fit)

    joblib.dump(behav_scaler, SCALER_PATH)
    print(f"✅ StandardScaler (behav_scaler) 已在 {SCALEABLE_BEHAV_FEATURES} 上拟合并保存。")
    print(f"拟合特征数量: {len(SCALEABLE_BEHAV_FEATURES)}")

except Exception as e:
    print(f"❌ 步骤 1 失败：StandardScaler 恢复失败。错误: {e}")
    exit()

# ======================================================================
# 步骤 2: 验证 Scaler 的拟合结果和预测输入是否匹配
# ======================================================================
print("\n--- 步骤 2: 验证 Scaler 拟合结果 ---")

# 检查 Scaler 期望的特征数量和名称是否匹配
scaler_fit_features = X_train_for_behav_fit.columns.tolist()

if scaler_fit_features != SCALEABLE_BEHAV_FEATURES:
    print("❌ 验证失败：Scaler 拟合特征顺序或名称不匹配！")
    print(f"Scaler 拟合的特征: {scaler_fit_features}")
    print(f"代码预期的特征: {SCALEABLE_BEHAV_FEATURES}")
    exit()
else:
    print("✅ 验证通过：Scaler 拟合特征列表与预期一致。")
    print(f"Scaler 拟合的 {len(scaler_fit_features)} 个特征: {scaler_fit_features}")

print("\n--- 诊断成功 ---")
print("如果以上步骤通过，则 Scaler 拟合和 Gradio 预测所需的数据切片应该匹配。")

--- 步骤 1: 恢复数据并拟合 StandardScaler ---
✅ StandardScaler (behav_scaler) 已在 ['studyhours'] 上拟合并保存。
拟合特征数量: 1

--- 步骤 2: 验证 Scaler 拟合结果 ---
✅ 验证通过：Scaler 拟合特征列表与预期一致。
Scaler 拟合的 1 个特征: ['studyhours']

--- 诊断成功 ---
如果以上步骤通过，则 Scaler 拟合和 Gradio 预测所需的数据切片应该匹配。


In [5]:
import gradio as gr
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- 路径常量 ---
PROJECT_ROOT = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/'
BASE_MODEL_DIR = os.path.join(PROJECT_ROOT, 'models')
FINAL_ENCODED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_encoded_data.csv')
FEATURES_LIST_PATH = os.path.join(PROJECT_ROOT, 'data/processed/final_feature_list.txt')

FINAL_STUDENT_MODEL_PATH = os.path.join(BASE_MODEL_DIR, 'nn_student_distilled.keras')
SCALER_PATH = os.path.join(BASE_MODEL_DIR, 'behav_scaler.pkl')

# ----------------------------------------------------------------------
# 核心辅助函数和变量
# ----------------------------------------------------------------------

def load_features(path):
    """从文件中读取准确的特征列表（小写）。"""
    with open(path, 'r') as f:
        features = [line.strip() for line in f if line.strip()]
    return features

def get_one_hot_col_name(category, value, feature_list):
    """根据Gradio的原始值，查找精确匹配的小写列名。"""
    normalized_value = value.lower()
    normalized_category = category.lower()
    target_name = f'{normalized_category}_{normalized_value}'

    match = next((col for col in feature_list if col == target_name), None)

    target_name_underscore = target_name.replace(' ', '_').replace('-', '_')
    if match is None:
        match = next((col for col in feature_list if col == target_name_underscore), None)

    return match

# ----------------------------------------------------------------------
# 步骤 1: 恢复 Gradio 运行环境
# ----------------------------------------------------------------------
print("--- 步骤 1: 恢复 Gradio 运行环境 ---")
try:
    tf.keras.backend.clear_session()

    # 从文件加载 BEHAV_FEATURES (黄金标准)
    BEHAV_FEATURES_FILE = load_features(FEATURES_LIST_PATH)
    SCALEABLE_BEHAV_FEATURES = ['studyhours'] # 确认只有这一个需要缩放

    # 恢复数据流程以获取 ALL_TRAINING_FEATURES (用于构造全零向量)
    df = pd.read_csv(FINAL_ENCODED_DATA_PATH)
    df.columns = df.columns.str.lower()
    X = df.drop(columns=['depression']).copy()
    y = df['depression'].copy()

    X_train_val, _, y_train_val, _ = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    val_size_relative = 0.15 / 0.85
    X_train, _, y_train, _ = train_test_split(X_train_val, y_train_val, test_size=val_size_relative, random_state=42, stratify=y_train_val)

    smote = SMOTE(random_state=42)
    X_train_full, _ = smote.fit_resample(X_train, y_train)
    ALL_TRAINING_FEATURES = X_train_full.columns.tolist()

    # 加载已保存的 behav_scaler 和模型
    behav_scaler = joblib.load(SCALER_PATH)
    model = load_model(FINAL_STUDENT_MODEL_PATH)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    FEATURE_DEFAULTS_RAW = dict(zip(ALL_TRAINING_FEATURES, [0.0] * len(ALL_TRAINING_FEATURES)))
    NON_SCALEABLE_BEHAV_FEATURES = [f for f in BEHAV_FEATURES_FILE if f not in SCALEABLE_BEHAV_FEATURES]

    print("✅ Gradio 依赖恢复成功。")

except Exception as e:
    print(f"❌ Gradio 环境恢复失败。错误: {e}")
    exit()

# --- Gradio 界面配置 ---
DIET_HABIT_OPTIONS = ['Healthy', 'Moderate', 'Unhealthy', 'Others']
SLEEP_DUR_OPTIONS = ['5-6 hours', '7-8 hours', 'Less than 5 hours', 'More than 8 hours', 'Others']

INPUT_FEATURE_NAMES = {
    'studyhours': '学习时长 (StudyHours)',
    'sleepdur': '睡眠时间 (SleepDur)',
    'diethabit': '饮食习惯 (DietHabit)'
}

# ----------------------------------------------------------------------
# 核心预测逻辑 (特征重组)
# ----------------------------------------------------------------------

def predict_depression(study_hours, sleep_dur, diet_habit):
    """ 接收用户输入的原始值，构造特征向量并预测。 """

    input_data_raw = {k: v for k, v in FEATURE_DEFAULTS_RAW.items()}

    # 1. 构造 X_pred_raw (全特征向量)
    input_data_raw['studyhours'] = study_hours

    matched_sleep_col = get_one_hot_col_name('SleepDur', sleep_dur, ALL_TRAINING_FEATURES)
    if matched_sleep_col: input_data_raw[matched_sleep_col] = 1

    matched_diet_col = get_one_hot_col_name('DietHabit', diet_habit, ALL_TRAINING_FEATURES)
    if matched_diet_col: input_data_raw[matched_diet_col] = 1

    X_pred_raw = pd.DataFrame([input_data_raw], columns=ALL_TRAINING_FEATURES)

    # 2. 缩放 (仅 studyhours)
    X_pred_scaleable = X_pred_raw[SCALEABLE_BEHAV_FEATURES] # 1 维 (studyhours)
    X_pred_scaled = behav_scaler.transform(X_pred_scaleable) # 缩放后的 studyhours

    # 3. 提取未缩放的 One-Hot 特征 (9 维)
    X_pred_non_scaleable = X_pred_raw[NON_SCALEABLE_BEHAV_FEATURES]

    # 4. 重新组合：创建最终的 10 维输入向量 (顺序必须匹配 BEHAV_FEATURES_FILE)

    final_data_list = []
    # 使用一个字典来存储缩放后的 studyhours，方便按顺序提取
    scaled_dict = dict(zip(SCALEABLE_BEHAV_FEATURES, X_pred_scaled.flatten()))

    for feature in BEHAV_FEATURES_FILE:
        if feature in SCALEABLE_BEHAV_FEATURES:
            # 提取缩放后的 studyhours
            final_data_list.append(scaled_dict[feature])
        else:
            # 提取未缩放的 One-Hot 特征值
            final_data_list.append(X_pred_non_scaleable[feature].iloc[0])

    X_final_input = np.array(final_data_list).reshape(1, 10) # 最终的 1x10 向量

    # 5. 预测
    prob = model.predict(X_final_input, verbose=0).flatten()[0]

    # 6. 结果输出
    if prob >= 0.5:
        prediction_text = "预测结果：抑郁风险较高"
        color = "red"
    else:
        prediction_text = "预测结果：抑郁风险较低"
        color = "green"

    return (
        f"<h2 style='color: {color};'> {prediction_text} </h2>"
        f"<h3> 预测概率: {prob*100:.2f}% </h3>"
        "<p>模型已部署。请尝试极端输入来验证模型决策。</p>"
    )

# ----------------------------------------------------------------------
# Gradio 界面设计
# ----------------------------------------------------------------------
iface = gr.Interface(
    fn=predict_depression,
    inputs=[
        gr.Slider(minimum=0, maximum=10, step=0.5, value=3.0, label=INPUT_FEATURE_NAMES['studyhours']),
        gr.Radio(choices=SLEEP_DUR_OPTIONS, label=INPUT_FEATURE_NAMES['sleepdur'], value='7-8 hours'),
        gr.Radio(choices=DIET_HABIT_OPTIONS, label=INPUT_FEATURE_NAMES['diethabit'], value='Moderate')
    ],
    outputs=gr.HTML(label="预测结果"),
    title="🧠 知识蒸馏 NN 抑郁风险预测演示 (最终特征重组版)",
    description="模型基于您的 10 个生活方式特征进行预测。已解决 Scaler 拟合特征不一致问题。",
    theme=gr.themes.Soft()
)

# 运行界面
print("\n--- 启动 Gradio 界面 ---")
iface.launch(share=True)

--- 步骤 1: 恢复 Gradio 运行环境 ---


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ Gradio 依赖恢复成功。

--- 启动 Gradio 界面 ---
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac5481aa254f262984.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
